# PAROL6 机械臂夹爪测试 (修复版)

这个Notebook用于测试PAROL6机械臂的夹爪控制功能，并修复了机械臂主体不显示的问题。

In [ ]:
# ========================================
# Cell 1 - 环境设置和库导入
# ========================================

import os
# 设置环境变量
os.environ['DISPLAY'] = ':1'
os.environ['MUJOCO_GL'] = 'egl'  # 使用EGL渲染
os.environ['PYOPENGL_PLATFORM'] = 'egl'
os.environ['EGL_DEVICE_ID'] = '0'

# 检查NVIDIA GPU
try:
    result = os.popen('nvidia-smi').read()
    if 'NVIDIA' in result:
        print('\NVIDIA GPU 已检测到')
        print('NVIDIA驱动信息:')
        print(result[:200] + '...' if len(result) > 200 else result)
    else:
        print('\NVIDIA GPU 未检测到')
except:
    print('\NVIDIA GPU 检测失败')

In [ ]:
# ========================================
# Cell 2 - 导入必要的库
# ========================================

import mujoco
import numpy as np
import time
from PIL import Image
import matplotlib.pyplot as plt

print('✅ 所有库导入成功！')
print(f'   MuJoCo版本: {mujoco.__version__}')

In [ ]:
# ========================================
# Cell 3 - 加载PAROL6模型（修复版本）
# ========================================

# 使用新的修复版XML文件
xml_path = '/home/wzy/lerobot-mujoco/asset/example_scene_parol6.xml'

print('📖 加载MuJoCo模型...')
model = mujoco.MjModel.from_xml_path(xml_path)
data = mujoco.MjData(model)

print('✅ 模型加载成功！')
print(f'   总关节数: {model.njnt}')
print(f'   总执行器: {model.nu}')
print(f'   总body数: {model.nbody}')
print(f'   摄像头数: {model.ncam}')

# 打印所有关节名称
print('\🔧 关节列表:')
for i in range(model.njnt):
    joint_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_JOINT, i)
    if joint_name:
        joint_type = 'revolute' if model.jnt_type[i] == 0 else 'slide'
        print(f'   {i}: {joint_name} ({joint_type})')

# 打印所有摄像头
print('\📷 摄像头列表:')
for i in range(model.ncam):
    cam_name = mujoco.mj_id2name(model, mujoco.mjtObj.mjOBJ_CAMERA, i)
    print(f'   {i}: {cam_name}')

In [ ]:
# ========================================
# Cell 4 - 渲染和可视化函数（增强版）
# ========================================

def render_camera(model, data, camera_name='agentview', width=640, height=480):
    '''
    从指定摄像头渲染图像
    
    Args:
        model: MuJoCo模型
        data: MuJoCo数据
        camera_name: 摄像头名称
        width: 图像宽度
        height: 图像高度
    
    Returns:
        numpy array: RGB图像 (H, W, 3)
    '''
    # 创建渲染器
    renderer = mujoco.Renderer(model, height=height, width=width)
    
    # 更新场景
    renderer.update_scene(data, camera=camera_name)
    
    # 渲染图像
    image = renderer.render()
    
    return image

def show_multi_view(model, data, save_path=None, test_name='test'):
    '''
    显示多视角图像，并可选择保存到本地
    
    Args:
        model: MuJoCo模型
        data: MuJoCo数据  
        save_path: 保存目录路径，如果为None则不保存
        test_name: 测试名称，用于文件名
    '''
    # 获取所有摄像头的图像
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    camera_names = ['agentview', 'gripper_cam', 'topview', 'sideview']
    
    for idx, cam_name in enumerate(camera_names):
        try:
            image = render_camera(model, data, camera_name=cam_name)
            ax = axes[idx // 2, idx % 2]
            ax.imshow(image)
            ax.set_title(f'{cam_name}', fontsize=14, fontweight='bold')
            ax.axis('off')
        except Exception as e:
            print(f'⚠️ 无法渲染摄像头 {cam_name}: {e}')
    
    plt.tight_layout()
    
    # 保存图片
    if save_path:
        os.makedirs(save_path, exist_ok=True)
        timestamp = time.strftime('%Y%m%d_%H%M%S')
        filename = f'{test_name}_{timestamp}.png'
        full_path = os.path.join(save_path, filename)
        plt.savefig(full_path, dpi=150, bbox_inches='tight')
        print(f'💾 图片已保存到: {full_path}')
    
    plt.show()

print('✅ 渲染函数定义完成！')

In [ ]:
# ========================================
# Cell 5 - 初始状态可视化（带保存）
# ========================================

# 重置到初始状态
mujoco.mj_resetData(model, data)

# 前向运动学
mujoco.mj_forward(model, data)

print('📸 初始状态多视角图像:')
show_multi_view(model, data, save_path='/home/wzy/lerobot-mujoco/test_img_output', test_name='initial_state')

# 打印初始关节位置
print('\🔧 初始关节位置:')
for i in range(model.nq):
    print(f'   qpos[{i}] = {data.qpos[i]:.4f}')

In [ ]:
# ========================================
# Cell 6 - 测试夹爪开合（闭合）
# ========================================

print('🤏 测试夹爪闭合...')

# 重置模型
mujoco.mj_resetData(model, data)

# 设置夹爪控制量
# data.ctrl[6] = 左手指 (rh_l1)
# data.ctrl[7] = 右手指 (rh_r1)
# 值范围：[0, 0.03]，0表示闭合，0.03表示完全打开

# 闭合夹爪（设置为0）
data.ctrl[6] = 0.0  # 左手指闭合
data.ctrl[7] = 0.0  # 右手指闭合

# 运行仿真让夹爪闭合
for _ in range(200):
    mujoco.mj_step(model, data)

print('✅ 夹爪已闭合')
print(f'   左手指位置: {data.qpos[6]:.4f}')
print(f'   右手指位置: {data.qpos[7]:.4f}')

# 显示闭合状态
print('\📸 夹爪闭合状态:')
show_multi_view(model, data, save_path='/home/wzy/lerobot-mujoco/test_img_output', test_name='gripper_closed')

In [ ]:
# ========================================
# Cell 7 - 测试夹爪开合（打开）
# ========================================

print('✋ 测试夹爪打开...')

# 重置模型
mujoco.mj_resetData(model, data)

# 打开夹爪（设置为最大值）
data.ctrl[6] = 0.03  # 左手指完全打开
data.ctrl[7] = 0.03  # 右手指完全打开

# 运行仿真让夹爪打开
for _ in range(200):
    mujoco.mj_step(model, data)

print('✅ 夹爪已打开')
print(f'   左手指位置: {data.qpos[6]:.4f}')
print(f'   右手指位置: {data.qpos[7]:.4f}')

# 显示打开状态
print('\📸 夹爪打开状态:')
show_multi_view(model, data, save_path='/home/wzy/lerobot-mujoco/test_img_output', test_name='gripper_open')

In [ ]:
# ========================================
# Cell 8 - 测试夹爪动态开合动画
# ========================================

print('🎬 测试夹爪开合动画...')

# 重置模型
mujoco.mj_resetData(model, data)

# 创建图像序列
images = []
n_frames = 60

# 生成开合动画
for i in range(n_frames):
    # 正弦波开合: 0 -> 0.03 -> 0
    t = i / n_frames * 2 * np.pi
    gripper_pos = 0.015 + 0.015 * np.sin(t)  # 范围 [0, 0.03]
    
    # 设置控制量
    data.ctrl[6] = gripper_pos
    data.ctrl[7] = gripper_pos
    
    # 运行仿真
    for _ in range(10):
        mujoco.mj_step(model, data)
    
    # 渲染图像
    image = render_camera(model, data, camera_name='gripper_cam', width=480, height=480)
    images.append(image)

print(f'✅ 生成了 {len(images)} 帧图像')

# 保存动画序列
os.makedirs('/home/wzy/lerobot-mujoco/test_img_output', exist_ok=True)
for i, img in enumerate(images):
    img_pil = Image.fromarray(img)
    img_pil.save(f'/home/wzy/lerobot-mujoco/test_img_output/animation_frame_{i:03d}.png')

print('💾 动画帧已保存到 /home/wzy/lerobot-mujoco/test_img_output/')

# 显示几个关键帧
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
frame_indices = [0, 15, 30, 45]

for idx, frame_idx in enumerate(frame_indices):
    axes[idx].imshow(images[frame_idx])
    axes[idx].set_title(f'Frame {frame_idx}', fontsize=12)
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('/home/wzy/lerobot-mujoco/test_img_output/gripper_animation_summary.png', dpi=150, bbox_inches='tight')
plt.show()

print('💡 提示：夹爪应该从闭合 -> 打开 -> 闭合循环')

In [ ]:
# ========================================
# Cell 9 - 测试机械臂运动+夹爪控制
# ========================================

print('🤖 测试机械臂运动配合夹爪控制...')

# 重置模型
mujoco.mj_resetData(model, data)

# 设置一些关节目标位置（弧度）
# 让机械臂做一些动作
target_positions = [0.5, -0.3, 0.8, -1.2, 0.6, 0.4]  # 6个关节的目标位置

# 逐步移动到新位置
for step in range(100):
    # 平滑插值到目标位置
    for i in range(6):
        current_error = target_positions[i] - data.qpos[i]
        data.ctrl[i] = current_error * 10  # PD控制
    
    # 同时控制夹爪开合
    gripper_cycle = np.sin(step * 0.1) * 0.015 + 0.015
    data.ctrl[6] = gripper_cycle
    data.ctrl[7] = gripper_cycle
    
    # 运行仿真
    mujoco.mj_step(model, data)

print('✅ 机械臂+夹爪协同控制完成')
print(f'   最终关节位置: {[data.qpos[i]:.3f for i in range(6)]}')
print(f'   最终夹爪位置: {data.qpos[6]:.4f}, {data.qpos[7]:.4f}')

# 显示最终状态
print('\📸 最终状态:')
show_multi_view(model, data, save_path='/home/wzy/lerobot-mujoco/test_img_output', test_name='arm_gripper_combined')

In [ ]:
# ========================================
# Cell 10 - 验收检查
# ========================================

print('🔍 执行验收检查...')

# 检查1: 模型加载
try:
    assert model is not None
    assert data is not None
    print('✅ 模型加载检查通过')
except:
    print('❌ 模型加载检查失败')

# 检查2: 关节数量
try:
    assert model.njnt >= 8  # 6个机械臂关节 + 2个夹爪关节
    print(f'✅ 关节数量检查通过: {model.njnt}个关节')
except:
    print(f'❌ 关节数量检查失败: 只有{model.njnt}个关节')

# 检查3: 执行器数量
try:
    assert model.nu >= 8  # 6个机械臂执行器 + 2个夹爪执行器
    print(f'✅ 执行器数量检查通过: {model.nu}个执行器')
except:
    print(f'❌ 执行器数量检查失败: 只有{model.nu}个执行器')

# 检查4: 摄像头
try:
    assert model.ncam >= 4  # 至少有4个摄像头
    print(f'✅ 摄像头检查通过: {model.ncam}个摄像头')
except:
    print(f'❌ 摄像头检查失败: 只有{model.ncam}个摄像头')

# 检查5: 关键body是否存在
expected_bodies = ['base_link', 'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'gripper_base']
missing_bodies = []
for body_name in expected_bodies:
    try:
        body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, body_name)
        if body_id == -1:
            missing_bodies.append(body_name)
    except:
        missing_bodies.append(body_name)

if not missing_bodies:
    print('✅ 关键body检查通过')
else:
    print(f'⚠️  缺失的body: {missing_bodies}')

print('\🎉 测试完成！')
print('\📁 所有图片已保存到: /home/wzy/lerobot-mujoco/test_img_output/')
print('\💡 如果机械臂主体仍然不显示，请检查:')
print('   1. XML文件是否正确加载')
print('   2. 渲染器设置是否正确')
print('   3. 摄像头位置和视角')
print('   4. 模型缩放比例是否正确')